# **DATA PREPROCESSING**

In [11]:
# Khai báo thư viện
import pandas as pd

import warnings 
warnings.filterwarnings('ignore')

Đọc dữ liệu từ file `train.csv` vào `df`

In [12]:
df = pd.read_csv('./train.csv').drop('Id', axis = 1)
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Xử lý dữ liệu bị mất

Ta sẽ xem xét những cột có dữ liệu bị mất mát

In [13]:
cnull = 0
for column in df:
    if df[column].isnull().values.any() == True:
        print(column, "has null value")
        cnull += 1
print(f"Số cột có dữ liệu bị thiếu là {cnull}")

LotFrontage has null value
Alley has null value
MasVnrType has null value
MasVnrArea has null value
BsmtQual has null value
BsmtCond has null value
BsmtExposure has null value
BsmtFinType1 has null value
BsmtFinType2 has null value
Electrical has null value
FireplaceQu has null value
GarageType has null value
GarageYrBlt has null value
GarageFinish has null value
GarageQual has null value
GarageCond has null value
PoolQC has null value
Fence has null value
MiscFeature has null value
Số cột có dữ liệu bị thiếu là 19


Sau khi tham khảo từ `data_description.txt` thì ta sẽ xử lý dữ liệu bị mất ở một vài cột dữ liệu sau

Dữ liệu thuộc dạng cột `Numerical`

In [14]:
### DỮ LIỆU THUỘC CỘT NUMERICAL    

# Fill Na values for Lot Frontage (numerical)
df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
# Fill Na values for MasVnrArea (numerical)
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)
# Fill Na values for GarageYrBlt (numerical)
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

Dữ liệu thuộc dạng cột `Categorical`

In [15]:
### DỮ LIỆU THUỘC CỘT CATEGORICAL

# Fill Na values for Alley (categorical)
df['Alley'] =  df['Alley'].fillna('NoAlleyAccess')
# Fill Na values for MasVnrType (categorical)
df['MasVnrType'] = df['MasVnrType'].fillna('None')
# Fill Na values for GarageFinish (categorical)
df['GarageFinish'] = df['GarageFinish'].fillna('NoGarage')
# Fill Na values for Electrical (categorical)
df['Electrical'] = df['Electrical'].fillna('None')
# Fill Na values for GarageType (categorical)
df['GarageType'] = df['GarageType'].fillna('NoGarage')
# Fill Na values for Fence (categorical)
df['Fence'] = df['Fence'].fillna('NoFence')
# Fill Na values for MiscFeature (categorical)
df['MiscFeature'] = df['MiscFeature'].fillna('None')

Tạo hàm chuyển từ Categorical sang Numerical

In [16]:
def Cat2Num(df, col, nominal = True):
    # Thuộc loại nominal
    if nominal == True:
        numerical = pd.get_dummies(df[col])
        try:
            col_name = col + '_' + numerical.keys()
            numerical.columns = col_name
        except: 
            True
        df = df.drop(columns = col)
        return pd.concat([df, numerical], axis= 'columns')
    # Thuộc loại ordinal
    else:
        ordinal = [['Ex', 'Gd', 'TA', 'Fa', 'Po', 'NA'],
                   ['Gd', 'Av', 'Mn', 'No', 'NA'],
                   ['GLQ', 'ALQ', 'BLQ', 'Rec', 'LWQ', 'Unf', 'NA'],
                   ['N', 'Y']]
        df[col] = df[col].fillna('NA')
        for o in ordinal:
            num = list(range(0, len(o), 1))
            num.sort(reverse= True) 
            df[col] = df[col].replace(o, num)
        
        return df

Từ file `data_description.txt` ta sẽ phân loại 2 loại categorical là `ordinal` và `nominal` 

In [17]:
categorical_nominal = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                       'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
                       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType', 'Foundation', 'Heating', 'Electrical', 
                       'Functional', 'GarageType', 'GarageFinish', 'PavedDrive',
                       'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
                       ]
categorical_ordinal = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
                       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
                       'HeatingQC', 'CentralAir', 'KitchenQual', 'FireplaceQu',
                       'GarageQual', 'GarageCond', 'PoolQC'
                       ]
df = Cat2Num(df, categorical_ordinal, False)
df = Cat2Num(df, categorical_nominal)

Dữ liệu sau khi tiền xử lý dữ liệu thiếu và chuyển hoá categorical sang numerical

In [18]:
df.shape

(1460, 247)

In [19]:
df.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,2003,196.0,4,3,...,0,0,0,1,0,0,0,0,1,0
1,20,80.0,9600,6,8,1976,1976,0.0,3,3,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2001,2002,162.0,4,3,...,0,0,0,1,0,0,0,0,1,0
3,70,60.0,9550,7,5,1915,1970,0.0,3,3,...,0,0,0,1,1,0,0,0,0,0
4,60,84.0,14260,8,5,2000,2000,350.0,4,3,...,0,0,0,1,0,0,0,0,1,0


In [20]:
cnull = 0
for column in df:
    if df[column].isnull().values.any() == True:
        print(column, "has null value")
        cnull += 1
print(f"Số cột có dữ liệu bị thiếu là {cnull}")

Số cột có dữ liệu bị thiếu là 0
